In [1]:
import azure.ai.ml

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential =  DefaultAzureCredential()
    credential.get_token("https://management.azure.com/default")
except Exception as ex:
    print("i am here")
    credential = InteractiveBrowserCredential()
    



DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.
	ManagedIdentityCredential: No token received.
To mitigate this issue, please refer to the troubleshooting guidelines here at https://aka.ms/azsdk/python/identity/defaultazurecredential/troubleshoot.


i am here


In [17]:
# i had to use cli credential with --no browser parameter , cause the redirect is having issues.

import azure.ai.ml

from azure.ai.ml import MLClient
from azure.identity import AzureCliCredential


credential= AzureCliCredential()

In [18]:
ml_client = MLClient(
    credential=credential,
    subscription_id="31241946-ad24-4495-85e2-31248c60d164",
    resource_group_name="ariefinternal",
    workspace_name="ariefmlworkspace"
)

Register and upload the car dataset


In [14]:
#this uploads the file from the local store to the default blob store under the default container and then LocalUpload/4*******/good_car_data.csv
#this gibberish is a problem with the 4***
# and the studio shows this LocalUpload/4a337dba56504a0bc17f6e0e49d1adb1/good_car_data.csv 

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
my_path = "abfss://amex@ariefgen2.dfs.core.windows.net/data/mlcardata"
car_cleaned_data = Data(
    path="../../data/good_car_data.csv",
    type=AssetTypes.URI_FILE,
    description="Good Car Data",
    name="car",
    version='1'
)

ml_client.data.create_or_update(car_cleaned_data)

Uploading good_car_data.csv (< 1 MB): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44.2k/44.2k [00:00<00:00, 1.69MB/s]


Data({'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'car', 'description': 'Good Car Data', 'tags': {}, 'properties': {}, 'id': '/subscriptions/31241946-ad24-4495-85e2-31248c60d164/resourceGroups/ariefinternal/providers/Microsoft.MachineLearningServices/workspaces/ariefmlworkspace/data/car/versions/1', 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2022_05_01.models._models_py3.SystemData object at 0x7f06a811e640>, 'serialize': <msrest.serialization.Serializer object at 0x7f06ab6cfa90>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/31241946-ad24-4495-85e2-31248c60d164/resourcegroups/ariefinternal/workspaces/ariefmlworkspace/datastores/workspaceblobstore/paths/LocalUpload/4a337dba56504a0bc17f6e0e49d1adb1/good_car_data.csv', 'referenced_uris': None})

In [26]:
#this registers where the location is shown.
my_path = "abfss://amex@ariefgen2.dfs.core.windows.net/data/mlcardata/good_car_data.csv"
car_cleaned_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="The Real car Data",
    name="goodcardata",
    version='1'
)

ml_client.data.create_or_update(car_cleaned_data)

NameError: name 'Data' is not defined

In [23]:
from azure.ai.ml.entities import AmlCompute
cpu_compute_target="amlcluster"

try:
    cpu_cluster =  ml_client.compute.get(cpu_compute_target)
    print(
        f"you already have a cluster named {cpu_compute_target}, we will resue it"
    )
except Exception:
    print("creating a new compute target")
    cpu_cluster=AmlCompute(
        name="amlcluster",
        type="aml-compute",
        size="STANDARD_D2_V2",
        min_instances=0,
        max_instances=2,
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated"
        )
    cpu_cluster=ml_client.begin_create_or_update(cpu_cluster)

print(
    f"AmlCompute with name {cpu_cluster.name} is created, the compute size is {cpu_cluster.size}"
)


you already have a cluster named amlcluster, we will resue it
AmlCompute with name amlcluster is created, the compute size is STANDARD_D2_V2


In [5]:
import os
dependencies_dir="../dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [20]:
%%writefile {dependencies_dir}/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scipy=1.7.1
  - scikit-learn=0.24.2
  - pandas>=1.1,<1.2
  - seaborn=0.11.2
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - xlrd==2.0.1
    - mlflow==1.26.0
    - azureml-mlflow==1.41.0

    


Overwriting ../dependencies/conda.yml


In [6]:
from azure.ai.ml.entities import Environment
custom_env_name = "arief-scikit-learn"
pipeline_job_env = Environment(
    name=custom_env_name,
    description="arief scikit learn environment",
    tags={"scikit-learn":"0.24.2","whose-environment":"arief_scikit_env"},
    conda_file=os.path.join(dependencies_dir,"conda.yml"),
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
    version="1.0.0"

)

In [7]:
pipeline_job_env

Environment({'is_anonymous': False, 'auto_increment_version': False, 'name': 'arief-scikit-learn', 'description': 'arief scikit learn environment', 'tags': {'scikit-learn': '0.24.2', 'whose-environment': 'arief_scikit_env'}, 'properties': {}, 'id': None, 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fdc5d1a6a30>, 'version': '1.0.0', 'latest_version': None, 'conda_file': OrderedDict([('name', 'model-env'), ('channels', ['conda-forge']), ('dependencies', ['python=3.8', 'numpy=1.21.2', 'pip=21.2.4', 'scipy=1.7.1', 'scikit-learn=0.24.2', 'pandas>=1.1,<1.2', OrderedDict([('pip', ['inference-schema[numpy-support]==1.3.0', 'xlrd==2.0.1', 'mlflow==1.26.0', 'azureml-mlflow==1.41.0'])])])]), 'image': 'mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest', 'build': None, 'inference_config': None, 'os_type': None, 'arm_type': 'environment_version', 'conda_file_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/arbavan1/c

In [9]:
pipeline_job_env =  ml_client.environments.create_or_update(pipeline_job_env)

Creating the first Component

In [11]:
print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name arief-scikit-learn is registered to workspace, the environment version is 1.0.0


In [2]:
import os
data_prep_src_dir="../components/data_prep"
os.makedirs(data_prep_src_dir, exist_ok=True)


In [10]:
%%writefile {data_prep_src_dir}/data_prep.py

import os
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
import logging
import mlflow
import seaborn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

def main():
    """Main function of the script
    got it"""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, help="split ratio", required=False, default=0.2)
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")

    args = parser.parse_args()

    #start logging
    mlflow.start_run()
    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))
    print("input data:",args.data)


    car= pd.read_csv(args.data,header=1, index_col=0)
    print("initial size data", car.shape)
    
    #remove any price greater than 6000000
    car=car[car['Price']<6000000]
    print("Size after removing rows with price greater than 6000000 ", car.shape)
     
    mlflow.log_metric("initial_num_samples", car.shape[0])
    mlflow.log_metric("initial_num_features", car.shape[1] - 1)

    X=car[['name','company','year','kms_driven','fuel_type']]
    y=car['Price'] 

    #train test split
    X_train,X_test,y_train,y_test=train_test_split(car,y, test_size=args.test_train_ratio)
   
    #get all the categories from all the data
    ohe=OneHotEncoder()
    ohe.fit(car[['name','company','fuel_type']])

    #make column transformer for all the categorical variables.
    column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']), remainder='passthrough')
    # fit transform and convert from sparse array
    X_train =column_trans.fit_transform(X_train).toarray()
    X_test = column_trans.fit_transform(X_test).toarray()
    
    #convert to dataframe and merge with price, please convert the series into dataframe by using to_frame() function
    train_df=pd.DataFrame(X_train)
    train_df.merge(y_train.to_frame(), left_index=True, right_index=True)
    test_df =pd.DataFrame(X_test)
    test_df.merge(y_test.to_frame(), left_index=True, right_index=True)

    #log the metrics
    mlflow.log_metric("TRANSFORMED_TRAIN_num_samples", train_df.shape[0])
    mlflow.log_metric("TRANSFORMED_TRAIN_num_features", train_df.shape[1] - 1)

    mlflow.log_metric("TRANSFORMED_TEST_num_samples", test_df.shape[0])
    mlflow.log_metric("TRANSFORMED_TEST_num_features", test_df.shape[1] - 1)

    # output paths are mounted as folder, therefore, we are adding a filename to the path
    train_df.to_csv(os.path.join(args.train_data, "data.csv"), index=False)
    test_df.to_csv(os.path.join(args.test_data, "data.csv"), index=False)

    mlflow.end_run()


if __name__ == "__main__":
    main()









Overwriting ../components/data_prep/data_prep.py


In [11]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

data_prep_component =  command(
    name="car_data_prep_components",
    display_name="Car Data Prepartion for training",
    description=" reads a csv file, uses one hot encoder and breaks into training and test data",
    inputs={
        "data": Input(type="uri_file"),
        "test_train_ratio": Input(type="number"),
    },
    outputs=dict(
        train_data=Output(type="uri_folder", mode="rw_mount"),
        test_data=Output(type="uri_folder", mode="rw_mount"),
    ),
    code=data_prep_src_dir,
    command=""" python data_prep.py \
        --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} \
        --train_data ${{outputs.train_data}} --test_data ${{outputs.test_data}} \
    """,
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}"
)

In [9]:
import os
train_src_dir = "../components/train"
os.makedirs(train_src_dir,exist_ok=True)

In [13]:
%%writefile {train_src_dir}/train.py
import argparse
import os
import pandas as pd
import mlflow
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


def select_first_file(path):
    """Selects first file in folder, use under assumption there is only one file in folder
    Args:
        path (str): path to directory or file to choose
    Returns:
        str: full path of selected file
    """
    files = os.listdir(path)
    return os.path.join(path, files[0])

# Start Logging
mlflow.start_run()

# enable autologging
mlflow.sklearn.autolog()

os.makedirs("./outputs", exist_ok=True)

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    parser.add_argument("--n_estimators", required=False, default=100, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    parser.add_argument("--registered_model_name", type=str, help="model name")
    parser.add_argument("--model", type=str, help="path to model file")
    args = parser.parse_args()

    train_df =  pd.read_csv(select_first_file(args.train_data))
    train_label=train_df["Price"]
    train_df=train_df.drop(columns=['Price'])
    
    test_df = pd.read_csv(select_first_file(args.test_data))
    test_label=test_df["Price"]
    test_df=test_df.drop(columns=['Price'])

    #convert into array
    X_train=train.df.values
    y_train=train_label.values

    X_test=test_df.values
    y_test=test_label.values

    #fit the model
    lr=LinearRegression()
    lr.fit(X_train,y_train)

    y_pred=lr.predict(X_test)
    print(r2_score(y_test,y_pred))
    
    #register the model to the workspace
    print("registering the model")
    mlflow.sklearn.log_model(
        sk_model=lr,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name,
    )

    #saving the model
    mlflow.sklearn.save_model(
        sk_model=lr,
        path=os.path.join(args.model,"trained_model"),
    )

    #stop logging
    mlflow.end_run()

if __name__ == "__main__":
    main()










Overwriting ../components/train/train.py


In [14]:
%%writefile {train_src_dir}/train.yml

name: train_car_prediction_model 
display_name: Train Car Prediction model 
type: command
inputs:
  train_data: 
    type: uri_folder
  test_data: 
    type: uri_folder
  registered_model_name:
    type: string
outputs:
  model:
    type: uri_folder
code: .
environment:
  # for this step, we'll use an AzureML curate environment
  azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:21
command: >-
  python train.py 
  --train_data ${{inputs.train_data}} 
  --test_data ${{inputs.test_data}} 
  --registered_model_name ${{inputs.registered_model_name}} 
  --model ${{outputs.model}}







Writing ../components/train/train.yml


In [15]:
from azure.ai.ml import load_component
train_component = load_component(path=os.path.join(train_src_dir,"train.yml"))


In [19]:
#register the component
train_component=ml_client.create_or_update(train_component)
print(
    f"Component {train_component.name} with Version {train_component.version} is registered"
)

Uploading train (0.0 MBs): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2857/2857 [00:00<00:00, 51865.80it/s]


Component train_car_prediction_model with Version 1 is registered


In [28]:
#create the pipeline from components

from azure.ai.ml import dsl, Input, Output
@dsl.pipeline(
    compute=cpu_compute_target,
    description="End To End Car Prediction Data Train Pipeline",
)
def car_prediction_pipeline(
    pipeline_job_data_input,
    pipeline_job_test_train_ratio,
    pipeline_job_registered_model_name,
):
    #data prep
    data_prep_job= data_prep_component(
        data=pipeline_job_data_input,
        test_train_ratio=pipeline_job_test_train_ratio,
    )

    #using train_func like python call with its own input
    train_job = train_component(
        train_data=data_prep_job.outputs.train_data, #using outputs from previous step
        test_data = data_prep_job.outputs.test_data, 
        registered_model_name=pipeline_job_registered_model_name,
    )

    #a pipeline returns a dict of outputs
    # keys will code for the pipeline output identifier
    return {
        "pipeline_job_train_data": data_prep_job.outputs.train_data,
        "pipeline_job_test_data": data_prep_job.outputs.test_data,
    }

In [29]:
registered_model_name="campusx_car_prediction_model"

#lets instantiate the pipeline
pipeline=  car_prediction_pipeline(
    pipeline_job_data_input=Input(type="uri_file", path=my_path),
    pipeline_job_test_train_ratio=0.2,
    pipeline_job_registered_model_name=registered_model_name,
)

In [30]:
import webbrowser

# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Project's name
    experiment_name="car_prediction_e2e_registered_components",
)
# open the pipeline in web browser
webbrowser.open(pipeline_job.services["Studio"].endpoint)

Uploading data_prep (0.0 MBs): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3385/3385 [00:00<00:00, 36374.47it/s]


True